In [ ]:
###
import numpy as np 
import pandas as pd 
import re
import random
import math
from tqdm.notebook import tqdm
from collections import Counter
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import random
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import pickle
import scipy
import sklearn
from keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras.models import Model

In [ ]:
#################
#### DATA #######
#################


df = pd.read_csv('full_2020_bow.csv') ## Full data available on request


# Chronological sorting of articles
df['date'] = pd.to_datetime(df['article_time'])
df = df.sort_values(by='date')

import random
random.seed(1234)
article_ids = np.array(df.article_id)
articles = np.unique(article_ids)

# Splitting at the halfway point
set1 = articles[:1476]
set2 = articles[1477:]



# Getting row indices for each set of articles in original data
index1 = []
index2 = []

for row in df.itertuples():
    if row.article_id in set1:
        index1.append(row.Index)
    else:
        index2.append(row.Index)
        
df1 = df.loc[index1,:] #### DATASET 1: for train, val, test
df2 = df.loc[index2,:] #### DATASET 2: for unseen eval articles


## Tokenizer for biLSTM and CNN
max_words = 10000  
tokenizer = Tokenizer(num_words=max_words)

all_texts = list(df.content)
tokenizer.fit_on_texts(all_texts)

In [ ]:
##########
## MODEL #
##########

with open('', 'rb') as f: ## fill in model
    model = pickle.load(f)

In [ ]:
#####################
### RANKING #########
#####################

# This notebook contains different cells for ranking depending on the variable set used by the model
# First cell runs code to rank based on models not trained on textual data: RF, SVM(correct_cols excludes BoW features)
# Second cell runs code to rank Rf_BoW, it keeps BoW columns (correct_cols to make sure feature set matches the trained model)
# third cell runs baseline
# fourth cell runs CNN and biLSTM
# fifth RobBERT

In [ ]:
# RF, SVM

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import collections
import pandas as pd
from openpyxl import load_workbook
import glob
import math
import argparse 
import numpy as np
from sklearn.metrics import ndcg_score


at_3 = []
at_5 = []
at_10 = []

prec_5 = []

correct_cols = [
 'reply_count',
 'respect_count',
 #'featured',
 'delta_minutes',
 'featured_posts_user',
 'rejected_count_user',
 'total_respect_user',
 'total_posts_user',
 'total_received_responses_user',
 'ratio_rejected',
 'ratio_featured',
 'ratio_reply',
 'ratio_respect',
 'wordcount',
 'intro_dist',
 'centr_dist',
 'avg_wordcount_user'
]

articles = list(set(df2.article_id))

for art in articles:
    nf = []
    f = []
    df_now = df2[df2.article_id==art]
    
    labels = df_now.featured
    content = df_now.content
    labels = list(labels)
    df_now = df_now[df_now.columns.intersection(correct_cols)]
    df_now = df_now.fillna(0)
    
    pred = model.predict_proba(df_now)
    pred = pd.DataFrame(pred)
    f = []
    nf = []
    pred.rename({0: 'nf', 1: 'f'}, axis=1, inplace=True)
    for row in pred.itertuples():
        if row.nf>0.5:
            nf.append(1)
            f.append(0)
        else:
            nf.append(0)
            f.append(1)
    labels2 = []
    for lab in labels:
        if lab == True:
            labels2.append(1)
        else:
            labels2.append(0)
    pred['pred_label']= f
    pred['label'] = labels2
    if 1 not in f:
        continue
    if 1 not in labels2:
        continue        
    pred = pred.sort_values(by=['f'], ascending=False)

    pred2 = list(pred['pred_label'])
    true2 = pred['label']
    labs = list(pred['label'])
    if len(labs)<10:
        continue
    for k in [3,5,10]:
        labs2 = labs[:k]
        preds2_temp = pred2[:k]
        if k == 3:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))
            if idcg ==0:
                at_3.append(0)
            else:
                ndcg = dcg/idcg
                at_3.append(ndcg)
        if k ==5:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))+(labs[3]/np.log2(4+1))+(labs[4]/np.log2(5+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))+(sorte[3]/np.log2(4+1))+(sorte[4]/np.log2(5+1))
            if idcg ==0:
                at_5.append(0)
            else:
                ndcg = dcg/idcg
                at_5.append(ndcg)
            prec_5.append(precision_score(labs2, preds2_temp, pos_label=1))

        if k ==10:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))+(labs[3]/np.log2(4+1))+(labs[4]/np.log2(5+1))+(labs[5]/np.log2(6+1))+(labs[6]/np.log2(7+1))+(labs[7]/np.log2(8+1))+(labs[8]/np.log2(9+1))+(labs[9]/np.log2(10+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))+(sorte[3]/np.log2(4+1))+(sorte[4]/np.log2(5+1))+(sorte[5]/np.log2(6+1))+(sorte[6]/np.log2(7+1))+(sorte[7]/np.log2(8+1))+(sorte[8]/np.log2(9+1))+(sorte[9]/np.log2(10+1))
            if idcg ==0:
                at_10.append(0)
            else:
                ndcg = dcg/idcg
                at_10.append(ndcg)
    
print('at3', s.mean(at_3),'at5', s.mean(at_5),'at10',s.mean(at_10))    

In [ ]:
correct_cols = [
 'reply_count',
 'respect_count',
 'featured',
 'delta_minutes',
 'featured_posts_user',
 'rejected_count_user',
 'total_respect_user',
 'total_posts_user',
 'total_received_responses_user',
 'ratio_rejected',
 'ratio_featured',
 'ratio_reply',
 'ratio_respect',
 'wordcount',
 'intro_dist',
 'centr_dist',
 'avg_wordcount_user',
    '000',
 '10',
 '100',
 '20',
 'aan het',
 'aan te',
 'aantal',
 'achter',
 'acties',
 'af',
 'afgelopen',
 'afstand',
 'al die',
 'alleen maar',
 'allemaal',
 'alles',
 'als de',
 'als een',
 'als er',
 'als het',
 'als ik',
 'als je',
 'als we',
 'als ze',
 'altijd',
 'ander',
 'anderen',
 'anders',
 'artikel',
 'auto',
 'bedrijf',
 'bedrijven',
 'beetje',
 'beleid',
 'bent',
 'besmet',
 'besmettingen',
 'best',
 'beste',
 'betalen',
 'beter',
 'betreft',
 'bevolking',
 'bezig',
 'bij de',
 'bijna',
 'bijvoorbeeld',
 'binnen',
 'blijft',
 'blijkbaar',
 'blijven',
 'boer',
 'br br',
 'br de',
 'br div',
 'br div div',
 'buiten',
 'burgers',
 'china',
 'cijfers',
 'corona',
 'covid',
 'daarnaast',
 'daarom',
 'dag',
 'dagen',
 'dan de',
 'dan is',
 'dan ook',
 'dat dit',
 'dat een',
 'dat er',
 'dat hij',
 'dat ik',
 'dat je',
 'dat niet',
 'dat we',
 'dat ze',
 'de maatregelen',
 'de meeste',
 'de mensen',
 'de natuur',
 'de overheid',
 'de politie',
 'de regels',
 'de regering',
 'de rest',
 'de wereld',
 'de zorg',
 'deel',
 'denk',
 'denk dat',
 'denken',
 'denkt',
 'dicht',
 'die de',
 'die het',
 'dingen',
 'dit is',
 'dit soort',
 'div br',
 'div br div',
 'div div',
 'div div br',
 'doden',
 'doe',
 'doet',
 'door de',
 'druk',
 'duidelijk',
 'duitsland',
 'echter',
 'economie',
 'een beetje',
 'een paar',
 'eerder',
 'eerst',
 'eerste',
 'eigen',
 'eigenlijk',
 'elkaar',
 'elke',
 'en als',
 'en dan',
 'en dat',
 'en die',
 'en een',
 'en het',
 'en ik',
 'en niet',
 'enige',
 'enkel',
 'enkele',
 'er een',
 'er geen',
 'er is',
 'er niet',
 'er nog',
 'er ook',
 'er zijn',
 'erg',
 'etc',
 'eten',
 'eu',
 'europa',
 'even',
 'extra',
 'feit',
 'ga',
 'gebeuren',
 'gebruiken',
 'gedaan',
 'gedrag',
 'geeft',
 'gehad',
 'geld',
 'geleden',
 'gelijk',
 'gelukkig',
 'gemaakt',
 'genoeg',
 'geval',
 'geven',
 'geweest',
 'gezegd',
 'gezien',
 'goede',
 'graag',
 'griep',
 'groep',
 'grond',
 'groot',
 'grootste',
 'grote',
 'haar',
 'had',
 'hadden',
 'halen',
 'hand',
 'hard',
 'heb ik',
 'heb je',
 'hebben we',
 'hebt',
 'heel veel',
 'helaas',
 'hele',
 'helemaal',
 'helemaal niet',
 'helpen',
 'hem',
 'het aantal',
 'het een',
 'het gaat',
 'het kabinet',
 'het land',
 'het niet',
 'het ook',
 'het probleem',
 'het rivm',
 'het virus',
 'het wel',
 'hij',
 'hoeveel',
 'hoop',
 'hoor',
 'houden',
 'huis',
 'ic',
 'idee',
 'ieder',
 'iedereen',
 'iemand',
 'iets',
 'ik ben',
 'ik denk',
 'ik denk dat',
 'ik heb',
 'ik het',
 'ik vind',
 'in een',
 'in het',
 'in nederland',
 'inderdaad',
 'is dan',
 'is dat',
 'is de',
 'is een',
 'is en',
 'is er',
 'is geen',
 'is niet',
 'is ook',
 'is voor',
 'ja',
 'jaar',
 'jaren',
 'je dat',
 'je de',
 'je een',
 'je het',
 'je niet',
 'jij',
 'juist',
 'jullie',
 'kabinet',
 'kan je',
 'kans',
 'kant',
 'keer',
 'kijk',
 'kijken',
 'kinderen',
 'kleine',
 'klopt',
 'kom',
 'komt',
 'krijgen',
 'krijgt',
 'kun',
 'kun je',
 'kunt',
 'laat',
 'laatste',
 'land',
 'landen',
 'lang',
 'laten',
 'laten we',
 'lees',
 'lekker',
 'leven',
 'liggen',
 'ligt',
 'lijkt',
 'lockdown',
 'lopen',
 'maakt',
 'maanden',
 'maar dat',
 'maar de',
 'maar een',
 'maar het',
 'maar ik',
 'maatregelen',
 'mag',
 'maken',
 'man',
 'manier',
 'me',
 'media',
 'meer dan',
 'meeste',
 'men',
 'mening',
 'mensen die',
 'met de',
 'met een',
 'met het',
 'meter',
 'mijn',
 'miljoen',
 'minder',
 'misschien',
 'moet je',
 'mogelijk',
 'mogen',
 'moment',
 'mondkapje',
 'mondkapjes',
 'mooi',
 'na',
 'naar de',
 'naar het',
 'namelijk',
 'natuur',
 'natuurlijk',
 'nederlandse',
 'nee',
 'nemen',
 'net',
 'net als',
 'niemand',
 'niet aan',
 'niet alleen',
 'niet dat',
 'niet de',
 'niet eens',
 'niet in',
 'niet meer',
 'niet op',
 'niet te',
 'niet voor',
 'niet zo',
 'niets',
 'nieuwe',
 'nieuws',
 'niks',
 'nl',
 'nodig',
 'nog een',
 'nog niet',
 'nog steeds',
 'nooit',
 'normaal',
 'nou',
 'om de',
 'om een',
 'om het',
 'om te',
 'onder',
 'onderzoek',
 'ons',
 'onze',
 'onzin',
 'ook de',
 'ook een',
 'ook niet',
 'ook nog',
 'op een',
 'op het',
 'op te',
 'open',
 'oplossing',
 'over de',
 'overheid',
 'paar',
 'partijen',
 'pas',
 'per',
 'plaats',
 'politie',
 'politiek',
 'praten',
 'precies',
 'prima',
 'probleem',
 'problemen',
 'reactie',
 'reden',
 'regels',
 'regering',
 'rest',
 'risico',
 'rivm',
 'rusland',
 'rutte',
 'scholen',
 'situatie',
 'snap',
 'snel',
 'soort',
 'staan',
 'staat',
 'steeds',
 'stemmen',
 'stikstof',
 'stoppen',
 'straks',
 'te doen',
 'te gaan',
 'te houden',
 'te maken',
 'te zijn',
 'tegen',
 'ten',
 'terug',
 'terwijl',
 'testen',
 'thuis',
 'tijd',
 'toe',
 'toen',
 'totaal',
 'trump',
 'tussen',
 'twee',
 'uit de',
 'uitstoot',
 'uw',
 'vaak',
 'vaccin',
 'valt',
 'van deze',
 'van een',
 'van het',
 'vanaf',
 'vanuit',
 'vast',
 'veel mensen',
 'vele',
 'verder',
 'vind',
 'vinden',
 'virus',
 'vlees',
 'vol',
 'volgens',
 'volgens mij',
 'voor een',
 'voor het',
 'vooral',
 'vraag',
 'waarom',
 'waarschijnlijk',
 'wanneer',
 'want',
 'waren',
 'wat een',
 'wat er',
 'week',
 'weet',
 'weg',
 'weinig',
 'weken',
 'wel een',
 'welke',
 'werd',
 'wereld',
 'werk',
 'werken',
 'werkt',
 'wet',
 'weten',
 'wie',
 'wij',
 'wil',
 'willen',
 'wilt',
 'word',
 'zaken',
 'zal',
 'ze niet',
 'zeer',
 'zeg',
 'zeggen',
 'zegt',
 'zeker',
 'zelfs',
 'zetten',
 'zie',
 'ziek',
 'ziekenhuis',
 'zien',
 'ziet',
 'zij',
 'zijn de',
 'zijn en',
 'zijn er',
 'zin',
 'zit',
 'zitten',
 'zoals',
 'zodat',
 'zonder',
 'zorg',
 'zorgen',
 'zouden',
 'zoveel',
 'zullen'
]


In [ ]:
## RF_BoW

import statistics as s

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import collections
import pandas as pd
from openpyxl import load_workbook
import glob
import math
import argparse 
import numpy as np
from sklearn.metrics import ndcg_score

plt.style.use('seaborn-whitegrid')

at_3 = []
at_5 = []
at_10 = []

prec_5 = []
prec_3 = []


articles = list(set(df2.article_id))

for art in articles:
    nf = []
    f = []
    df_now = df2[df2.article_id==art]
    
    labels = df_now.featured
    content = df_now.content
    topics = list(set(list(df_now.preferred_section_id)))
    labels = list(labels)
    df_now = df_now[df_now.columns.intersection(correct_cols)]
    df_now = df_now.fillna(0)
    
    pred = model.predict_proba(df_now)
    pred = pd.DataFrame(pred)
    f = []
    nf = []
    pred.rename({0: 'nf', 1: 'f'}, axis=1, inplace=True)
    for row in pred.itertuples():
        if row.nf>0.5:
            nf.append(1)
            f.append(0)
        else:
            nf.append(0)
            f.append(1)
    labels2 = []
    for lab in labels:
        if lab == True:
            labels2.append(1)
        else:
            labels2.append(0)
    pred['pred_label']= f
    pred['label'] = labels2
    if 1 not in f:
        continue
    if 1 not in labels2:
        continue        
    pred = pred.sort_values(by=['f'], ascending=False)

    pred2 = pred['pred_label']
    true2 = pred['label']
    labs = list(pred['label'])
    if len(labs)<10:
        continue
    if 1778528 in topics:
        topic.append('Covid-19')
    elif 1380734 in topics:
        topic.append('US election 2020')
    else:
        topic.append('Climate change')
    for k in [3,5,10]:
        labs2 = labs[:k]
        if k == 3:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))
            if idcg ==0:
                at_3.append(0)
            else:
                ndcg = dcg/idcg
                at_3.append(ndcg)
            prec_3.append(precision_score(labs2, preds2_temp, pos_label=1))
            
        if k ==5:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))+(labs[3]/np.log2(4+1))+(labs[4]/np.log2(5+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))+(sorte[3]/np.log2(4+1))+(sorte[4]/np.log2(5+1))
            if idcg ==0:
                at_5.append(0)
            else:
                ndcg = dcg/idcg
                at_5.append(ndcg)
            prec_5.append(precision_score(labs2, preds2_temp, pos_label=1))
        if k ==10:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))+(labs[3]/np.log2(4+1))+(labs[4]/np.log2(5+1))+(labs[5]/np.log2(6+1))+(labs[6]/np.log2(7+1))+(labs[7]/np.log2(8+1))+(labs[8]/np.log2(9+1))+(labs[9]/np.log2(10+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))+(sorte[3]/np.log2(4+1))+(sorte[4]/np.log2(5+1))+(sorte[5]/np.log2(6+1))+(sorte[6]/np.log2(7+1))+(sorte[7]/np.log2(8+1))+(sorte[8]/np.log2(9+1))+(sorte[9]/np.log2(10+1))
            if idcg ==0:
                at_10.append(0)
            else:
                ndcg = dcg/idcg
                at_10.append(ndcg)
    
print('at3', s.mean(at_3),'at5', s.mean(at_5),'at10',s.mean(at_10))    
print('Precision scores @: at3', s.mean(prec_3), 'at5', s.mean(prec_5))    

In [ ]:
## BASELINE 

import statistics as s

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import collections
import pandas as pd
from openpyxl import load_workbook
import glob
import math
import argparse 
import numpy as np
from sklearn.metrics import ndcg_score

at_3 = []
at_5 = []
at_10 = []
prec_5 = []
prec_3 = []

articles = list(set(df2.article_id))

for art in articles:
    nf = []
    f = []
    df_now = df2[df2.article_id==art]
    
    labels = df_now.featured
    labels = list(labels)
    df_now = df_now[df_now.columns.intersection(correct_cols)]
    df_now = df_now[order]
    df_now = df_now.fillna(0)
    rfeat = list(df_now.ratio_featured)
    f = []
    nf = []
    pred.rename({0: 'nf', 1: 'f'}, axis=1, inplace=True)
    for i in range(len(rfeat)):
        if rfeat[i]>0.03:
            nf.append(1)
            f.append(0)
        else:
            nf.append(0)
            f.append(1)
    labels2 = []
    for lab in labels:
        if lab == True:
            labels2.append(1)
        else:
            labels2.append(0)
    pred = pd.DataFrame(rfeat)
    pred['pred_label']=nf
    pred['label'] = labels2
    if 1 not in f:
        continue
    if 1 not in labels2:
        continue        
    pred = pred.sort_values(by=[0], ascending=False)

    pred2 = pred['pred_label']
    true2 = pred['label']
    labs = list(pred['label'])
    if len(labs)<10:
        continue
    for k in [3,5,10]:
        labs2 = labs[:k]
        if k == 3:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))
            if idcg ==0:
                at_3.append(0)
            else:
                ndcg = dcg/idcg
                at_3.append(ndcg)
            prec_3.append(precision_score(labs2, preds2_temp, pos_label=1))

        if k ==5:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))+(labs[3]/np.log2(4+1))+(labs[4]/np.log2(5+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))+(sorte[3]/np.log2(4+1))+(sorte[4]/np.log2(5+1))
            if idcg ==0:
                at_5.append(0)
            else:
                ndcg = dcg/idcg
                at_5.append(ndcg)
            prec_5.append(precision_score(labs2, preds2_temp, pos_label=1))

        if k ==10:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))+(labs[3]/np.log2(4+1))+(labs[4]/np.log2(5+1))+(labs[5]/np.log2(6+1))+(labs[6]/np.log2(7+1))+(labs[7]/np.log2(8+1))+(labs[8]/np.log2(9+1))+(labs[9]/np.log2(10+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))+(sorte[3]/np.log2(4+1))+(sorte[4]/np.log2(5+1))+(sorte[5]/np.log2(6+1))+(sorte[6]/np.log2(7+1))+(sorte[7]/np.log2(8+1))+(sorte[8]/np.log2(9+1))+(sorte[9]/np.log2(10+1))
            if idcg ==0:
                at_10.append(0)
            else:
                ndcg = dcg/idcg
                at_10.append(ndcg)
    
print('at3', s.mean(at_3),'at5', s.mean(at_5),'at10',s.mean(at_10))    
print('Precision scores @: at3', s.mean(prec_3), 'at5', s.mean(prec_5))    

In [ ]:
### CNN or biLSTM

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import collections
import pandas as pd
from openpyxl import load_workbook
import glob
import math
import argparse 
import numpy as np
from sklearn.metrics import ndcg_score
import statistics as s

at_3 = []
at_5 = []
at_10 = []

prec_5 = []
prec_3 = []


articles = list(set(df2.article_id))

for art in articles:
    nf = []
    f = []
    df_now = df2[df2.article_id==art]
    
    labels = df_now.featured
    content = list(df_now.content)
    labels = list(labels)

    art_seq = tokenizer.texts_to_sequences(content)
    max_sequence_length = 100  
    padded_sequences_art = pad_sequences(art_seq, maxlen=max_sequence_length)

    predictions = model.predict(padded_sequences_art)
    preds = []
    for i in range(len(predictions)):
        if predictions[i]>0.5:
            preds.append(1)
        else:
            preds.append(0)
    
    labels2 = []
    for lab in labels:
        if lab == True:
            labels2.append(1)
        else:
            labels2.append(0)

    

    if 1 not in preds:
        print('none pred')
        continue
    if 1 not in labels2:
        print('none real')
        continue      
        
    pred = pd.DataFrame(predictions)
    pred.columns=['feat']
    pred['non_feat'] = 1- pred.feat
    pred['pred_label'] = preds
    pred['label'] = labels2
    pred = pred.sort_values(by=['feat'], ascending=False)

    pred2 = list(pred['pred_label'])
   
    true2 = pred['label']
    labs = list(pred['label'])
    if len(labs)<10:
        continue
    for k in [3,5,10]:
        labs2 = labs[:k]
        preds2_temp = pred2[:k]
        if k == 3:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))
            if idcg ==0:
                at_3.append(0)
            else:
                ndcg = dcg/idcg
                at_3.append(ndcg)
            prec_3.append(precision_score(labs2, preds2_temp, pos_label=1))
        if k ==5:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))+(labs[3]/np.log2(4+1))+(labs[4]/np.log2(5+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))+(sorte[3]/np.log2(4+1))+(sorte[4]/np.log2(5+1))
            if idcg ==0:
                at_5.append(0)
            else:
                ndcg = dcg/idcg
                at_5.append(ndcg)
            prec_5.append(precision_score(labs2, preds2_temp, pos_label=1))

        if k ==10:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))+(labs[3]/np.log2(4+1))+(labs[4]/np.log2(5+1))+(labs[5]/np.log2(6+1))+(labs[6]/np.log2(7+1))+(labs[7]/np.log2(8+1))+(labs[8]/np.log2(9+1))+(labs[9]/np.log2(10+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))+(sorte[3]/np.log2(4+1))+(sorte[4]/np.log2(5+1))+(sorte[5]/np.log2(6+1))+(sorte[6]/np.log2(7+1))+(sorte[7]/np.log2(8+1))+(sorte[8]/np.log2(9+1))+(sorte[9]/np.log2(10+1))
            if idcg ==0:
                at_10.append(0)
            else:
                ndcg = dcg/idcg
                at_10.append(ndcg)
    
print('NDCG scores: at3', s.mean(at_3),'at5', s.mean(at_5),'at10',s.mean(at_10))   
print('Precision scores @: at3', s.mean(prec_3), 'at5', s.mean(prec_5))    

In [ ]:
## RobBERT

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import collections
import pandas as pd
from openpyxl import load_workbook
import glob
import math
import argparse 
import numpy as np
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer
from transformers import Trainer, TrainingArguments
import numpy as np
import random

tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robBERT-base")
at_3 = []
at_5= []
at_10=[]
prec_3 = []
prec_5 = []

target_names = ['Not_feat', 'Feat']

model = RobertaForSequenceClassification.from_pretrained("",num_labels=len(target_names)) ## select model


def get_prediction(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=205, return_tensors="pt")
    outputs = model(**inputs)
    probs = outputs[0].softmax(1)
    return probs.tolist()



used_arts = []
articles = list(set(df2.article_id))

for art in articles:
    if art in used_arts:
        print('already done')
        continue
    
    nf = []
    f = []
    df_now = df2[df2.article_id==art]
    df_feat = df_now[df_now.featured==True]

    text = list(df_now.content)
    labels = df_now.featured

    for x in range(len(labels)):
        if labels[x]==False:
            labels[x]='Not_feat'
        else:
            labels[x]='Feat'
    #print('Got the labels')
    for p in range(len(text)):
        temp = get_prediction(text[p])
        temp = temp[0]
        nf.append(temp[0])
        f.append(temp[1])
        #print(p)
    pred = pd.DataFrame(nf)
    pred['featured'] = f
    pred = pred.rename(columns={0: 'Not_featured', 'featured': 'Featured'})
    pred_label = []
    for row in pred.itertuples():
        if row.Not_featured  > row.Featured:
            pred_label.append('Not_Featured')
        else:
            pred_label.append('Featured')
        
    pred['Pred_label'] = pred_label
    pred['Label'] = labels
    pred = pred.sort_values(by=['Featured'], ascending=False)

    if 'Featured' not in pred_label:
        continue
    if 'Feat' not in labels:
        continue
    pred = pred.sort_values(by=['Featured'], ascending=False)
    pred2 = []
    true2 = []
    for row in pred.itertuples():
        if row.Pred_label=='Featured':
            pred2.append(1)
        else:
            pred2.append(0)
        if row.Label=='Feat':
            true2.append(1)
        else:
            true2.append(0)

    labs = true2
    if len(pred)<10:
        continue
    for k in [3,5,10]:
        labs2 = labs[:k]
        if k == 3:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))
            if idcg ==0:
                at_3.append(0)
            else:
                ndcg = dcg/idcg
                at_3.append(ndcg)
            prec_3.append(precision_score(labs2, preds2_temp, pos_label=1))
                
        if k ==5:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))+(labs[3]/np.log2(4+1))+(labs[4]/np.log2(5+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))+(sorte[3]/np.log2(4+1))+(sorte[4]/np.log2(5+1))
            if idcg ==0:
                at_5.append(0)
            else:
                ndcg = dcg/idcg
                at_5.append(ndcg)
            prec_5.append(precision_score(labs2, preds2_temp, pos_label=1))
                
        if k ==10:
            dcg = (labs[0]/np.log2(1+1))+(labs[1]/np.log2(2+1))+(labs[2]/np.log2(3+1))+(labs[3]/np.log2(4+1))+(labs[4]/np.log2(5+1))+(labs[5]/np.log2(6+1))+(labs[6]/np.log2(7+1))+(labs[7]/np.log2(8+1))+(labs[8]/np.log2(9+1))+(labs[9]/np.log2(10+1))
            sorte = sorted(labs2, reverse=True)
            idcg = (sorte[0]/np.log2(1+1))+(sorte[1]/np.log2(2+1))+(sorte[2]/np.log2(3+1))+(sorte[3]/np.log2(4+1))+(sorte[4]/np.log2(5+1))+(sorte[5]/np.log2(6+1))+(sorte[6]/np.log2(7+1))+(sorte[7]/np.log2(8+1))+(sorte[8]/np.log2(9+1))+(sorte[9]/np.log2(10+1))
            if idcg ==0:
                at_10.append(0)
            else:
                ndcg = dcg/idcg
                at_10.append(ndcg)
    used_arts.append(art)
    
print('NDCG scores: at3', s.mean(at_3),'at5', s.mean(at_5),'at10',s.mean(at_10))   
print('Precision scores @: at3', s.mean(prec_3), 'at5', s.mean(prec_5))        